In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import re
from glob import glob


import read_nml as rnl

from scipy.ndimage import uniform_filter
from scipy.ndimage import gaussian_filter
import matplotlib.colors as mcolors


from Utils import numerical_utils as nuti
from Utils import MyConstants as Co
from Utils import MakePressures as MkP

print( Co.cpair())

cpair = Co.cpair()
rdair = Co.Rdair()
grav= Co.grav()

In [ ]:
attr_list = []

def grab_attrs(ds):
    # store a copy of attrs from this file
    attr_list.append(dict(ds.attrs))
    return ds




def handle_wild( n ):
    if (n==99):
        n='*'
    elif (n is None):
        n='*'
    return n

In [ ]:
#################
# set case name
#################

day,hour=None,None

#case='camsnap-Baseline01'
case='xy-Baseline-v3.0'
case='xy-testfrontg'
#case='xy-vortsrc-03'


#case='xympas-x00'
case='xympas-Aug-4-2016'
year,month,day,hour=2016,8, 99, 99

case='xympas-check'
year,month,day,hour=2016,8, 4, 99


#case='xy-x00'
#case='xy-Aug2004'
#year,month,day,hour=2004,8, 99, 99

day=handle_wild( day )
hour=handle_wild( hour )

if (hour != '*') and (day != '*' ):
    seconds=3_600 * hour
    f=f'/glade/derecho/scratch/juliob/archive/GW_UnitTest/{case}/{case}.h.{year:04}-{month:02}-{day:02}-{seconds:05}.nc'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
    title=f'{case}: {year:04}-{month:02}-{day:02}-{seconds:05}'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
elif (hour == '*') and (day != '*' ):
    f=f'/glade/derecho/scratch/juliob/archive/GW_UnitTest/{case}/{case}.h.{year:04}-{month:02}-{day:02}-*.nc'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
    title=f'{case}: {year:04}-{month:02}-{day:02}'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
elif (day=='*'):
    f=f'/glade/derecho/scratch/juliob/archive/GW_UnitTest/{case}/{case}.h.{year:04}-{month:02}-*.nc'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
    title=f'{case}: {year:04}-{month:02}'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'





files = sorted(glob(f))
print(len(files), "files found")

In [ ]:
print(files)


In [ ]:
"""
for m, f in enumerate(files):
    fout= f.replace("-e00", "-n00")
    print( f+ fout) 
    ds = xr.open_dataset(f)
    ds = ds.assign_coords(member=m)   # scalar coordinate
    ds["member"].attrs.update(long_name="ensemble member")
    ds.to_netcdf(fout )        # append/overwrite coord in-place (NetCDF4)
"""



XX=xr.open_dataset( files[0] )

In [ ]:
XX.time.values


In [ ]:
Xgw=xr.open_mfdataset( f,   preprocess=grab_attrs, data_vars='different', coords='different', compat='no_conflicts'  )
#Xgw=xr.open_mfdataset( f,   preprocess=grab_attrs, combine='by_coords' )


ftopo='/glade/work/juliob/Topo/NCARTopoJTB/cases/fv1x1_Sco100_GrnlAnt/output/fv1x1_gmted2010_modis_bedmachine_nc3000_Laplace0100_noleak_greenlndantarcsgh30fac2.50_20251009.nc'
Topo=xr.open_dataset( ftopo )

In [ ]:
#read GW.dat file

%run -i val_camsnap_block.py


In [ ]:
%%time

ny = Xgw.sizes["ny"]
nx = Xgw.sizes["nx"]

j = np.arange(ny).repeat(nx)
i = np.tile(np.arange(nx), ny)

X2 = (
    Xgw.assign_coords(_j=("ncol", j), _i=("ncol", i))     # mapping ncol -> (j,i)
     .set_index(ncol=("_j", "_i"))
     .unstack("ncol")
     .rename({"_j": "ny", "_i": "nx"})
     .assign_coords(ny=Xgw["lat_R"], nx=Xgw["lon_R"])        # put real coords on axes
)


X2['lat']=Xgw['lat_R']
X2['lon']=Xgw['lon_R']
Xgw=X2

In [ ]:
Xgw

In [ ]:
%%time

#tau_rdg=Xgw.TAU_RDG.values
tau_mmt=Xgw.TAU_MOVMTN.values
tau_diag_mmt=Xgw.TAU_DIAG_MOVMTN.values
tau_src_mmt=Xgw.TAU_SRC_MOVMTN.values
u=Xgw.U.values
ubm=Xgw.UBM.values
zeta=Xgw.ZETA.values
th=Xgw.TH.values
piln=Xgw.piln.values
rhoi=Xgw.rhoi.values
ni=Xgw.ni.values
nm=Xgw.nm.values

print( np.max( tau_mmt) )
print( np.max( tau_diag_mmt) )



In [ ]:
plt.contourf( nm[0,23,:,:] - nm[2,23,:,:]  )
plt.colorbar()

In [ ]:
plt.plot( tau_src_mmt[0,28,:,:].flatten() )

In [ ]:
plt.contourf( tau_src_mmt[0,28,:,:] - tau_src_mmt[2,28,:,:]  )
plt.colorbar()

In [ ]:
plt.contourf( tau_diag_mmt[0,23,:,:] - tau_diag_mmt[2,23,:,:]  )
plt.colorbar()

In [ ]:
plt.contourf( tau_mmt[0,23,:,:] - tau_mmt[2,23,:,:]  )
plt.colorbar()

In [ ]:

for t in np.arange(8):
    print( np.max( tau_mmt[t,23:25,:,:] ) )

In [ ]:
if (Xgw.calculation_type == 'xympas'):
    zlev=X.lev.values
    reso='3.75km'
else:
    zlev=-7_000.*np.log( X.lev.values /1_000.)
    reso='14km'

lat=X.lat.values
lon=X.lon.values

In [ ]:
%%time

u=Xgw.U.values
zeta=Xgw.ZETA.values

nplo=4
t,z,y,x = 27,25,112,40
#t,z,y,x = 4,25,112,40

print( Xgw.time.values[t] )



fig,axs = plt.subplots( 1, nplo , figsize = (nplo*10,8) )

p=0
ax=axs[p]
ax.plot( tau_mmt[t,1:,y,x] , zlev )

p=1
ax=axs[p]
ax.plot( u[t,:,y,x] , zlev )

p=2
ax=axs[p]
ax.plot( zeta[t,:,y,x] , zlev )





In [ ]:

print( Xgw.sizes )#[('ny','nx')])
nt,nz,ny,nx = Xgw.sizes['time'],  Xgw.sizes['level'],  Xgw.sizes['ny'],  Xgw.sizes['nx']

In [ ]:
## Rho scaling
if (Xgw.calculation_type == 'xympas'):
    pint=X.PINT.values
    pmid = 0.5*( pint[:,0:nz,:,:] + pint[:,1:nz+1,:,:] )
    te = X.T.values
    te_int = np.zeros( (nt,nz+1,ny,nx) )
    te_int[:,0,:]=te[:,0,:,:]
    te_int[:,nz,:]=te[:,nz-1,:,:]
    for z in np.arange( start=1,stop=nz ):
        te_int[:,z,:] = 0.5*( te[:,z-1,:] + te[:,z,:] )
    rho3d = pint / (rdair * te_int )
    rho_scl = rho3d
else:
    pmid,pint,delp =MkP.Pressure ( X=X , p_00=100_000., Gridkey='tzyx' )
    te = X.T.values
    RhoProxy = X.RhoProxy.values
    RhoProxy3D = np.broadcast_to(RhoProxy[None, :, None, None], (nt, nz, ny, nx))
    rho3d = pmid / (rdair * te )
    rho_scl = rho3d / RhoProxy3D

rho_epwp = rho_scl * epwp

In [ ]:
%%time
#tau_rdg = tau_rdg.reshape((nt,nz+1,ny,nx) )
#tau_mmt = tau_mmt.reshape((nt,nz+1,ny,nx) )

tau_tot = tau_rdg + tau_mmt

In [ ]:
tau_rdg_tv = np.average( tau_rdg, axis=0)
tau_mmt_tv = np.average( tau_mmt, axis=0)
tau_tot_tv = np.average( tau_tot, axis=0)

upwp_tv = np.average(upwp,axis=0)
epwp_tv = np.average(epwp,axis=0)
rho_epwp_tv = np.average(rho_epwp,axis=0)


htopo=Topo.PHIS.values/grav



In [ ]:
%%time
print(nt,nz)

tau_tot_sm=np.zeros( (nt,nz+1,ny,nx) )
trunc_k, sigma=10, 4.5

for t in np.arange( nt ):
    for z in np.arange( nz ):
        tau_tot_sm[t,z,:,:] = gaussian_filter(  tau_tot[t,z,:,:] , sigma=sigma,
                                   truncate=((trunc_k-1)/2)/sigma, mode="nearest")
        
#tau_tot_smooth = uniform_filter(   tau_tot_tv[z,:,:]  , size=7, mode="wrap")   # 3x3

tau_tot_sm_tv = np.average( tau_tot_sm, axis=0 )

In [ ]:
z=np.argmin( np.abs( zlev - 20_000. ) )

title_ = f'{case}: {title} , Z={zlev[z]/1000.:.0f} km '     #        camsnap-yaaaa.h.2004-06-15-21600.nc'


#flev=[0.0001,0.0002,0.0005,  0.001,0.002,0.005,  0.01,0.02,0.05,  0.1,0.2,0.5,   1.0  ]
flev=[0.0001,0.0002,0.0005,  0.001,0.0015,0.002,0.003,0.004,0.005,0.006,0.008,    0.01,0.015,0.02,0.03,0.04,0.05,0.06,  0.1,0.2,0.5,   1.0  ]
flev=0.1*np.asarray( flev )
print(flev)
cmapN='gist_ncar'

cmap = plt.cm.bwr  #gist_ncar  # Or any other colormap
cmap = plt.cm.gist_ncar # .plasma #gist_ncar  #gist_ncar  # Or any other colormap
norm = mcolors.BoundaryNorm(boundaries=flev, ncolors=cmap.N, clip=False)

topocolor='white' #'black'


fig,axs=plt.subplots( 1, 3, figsize=(30,6) )
ax=axs[0]
c = ax.contourf( lon, lat, tau_tot_tv[z,:,:],levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Param GW "+ r"$\tau$")
plt.colorbar( c )

ax=axs[1]
c = ax.contourf( lon, lat, tau_tot_sm_tv[z,:,:],levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Smoothed param GW "+ r"$\tau$")
plt.colorbar( c )


ax=axs[2]
c = ax.contourf( lon, lat, rho_epwp_tv[z,:,:] ,levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( f'Resolved GW {r"$\tau$"} in {reso} sim.')
plt.colorbar( c )

plt.suptitle( title_ , fontsize=24) 

print(X.lev[z].values)
print(zlev[z])


In [ ]:
z=np.argmin( np.abs( zlev - 25_000. ) )
t=3

title_ = f'{case}: {title} , Z={zlev[z]/1000.:.0f} km '     #        camsnap-yaaaa.h.2004-06-15-21600.nc'

tau_tot_smooth = uniform_filter(   tau_tot_tv[z,:,:]  , size=7, mode="wrap")   # 3x3

trunc_k, sigma=7, 1.5
tau_tot_smooth = gaussian_filter(  tau_tot[t,z,:,:] , sigma=sigma,
                           truncate=((trunc_k-1)/2)/sigma, mode="nearest")


#flev=[0.0001,0.0002,0.0005,  0.001,0.002,0.005,  0.01,0.02,0.05,  0.1,0.2,0.5,   1.0  ]
flev=[0.0001,0.0002,0.0005,  0.001,0.0015,0.002,0.003,0.004,0.005,0.006,0.008,    0.01,0.015,0.02,0.03,0.04,0.05,0.06,  0.1,0.2,0.5,   1.0  ]
flev=0.1*np.asarray( flev )
print(flev)
cmapN='gist_ncar'

cmap = plt.cm.bwr  #gist_ncar  # Or any other colormap
cmap = plt.cm.gist_ncar # .plasma #gist_ncar  #gist_ncar  # Or any other colormap
norm = mcolors.BoundaryNorm(boundaries=flev, ncolors=cmap.N, clip=False)

topocolor='white' #'black'


fig,axs=plt.subplots( 1, 3, figsize=(30,6) )
ax=axs[0]
c = ax.contourf( lon, lat, tau_tot[t,z,:,:],levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Param GW "+ r"$\tau$")
plt.colorbar( c )

ax=axs[1]
c = ax.contourf( lon, lat, tau_tot_smooth,levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Smoothed param GW "+ r"$\tau$")
plt.colorbar( c )


ax=axs[2]
c = ax.contourf( lon, lat, rho_epwp[t,z,:,:] ,levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( f'Resolved GW {r"$\tau$"} in {reso} sim.')
plt.colorbar( c )

plt.suptitle( title_ , fontsize=24)

print(X.lev[z].values)
print(zlev[z])


In [ ]:
z0,lat0,lon0 = 25_000., -45.,10.
z,y,x = np.argmin( np.abs( zlev - z0 ) ),np.argmin( np.abs( lat-lat0 ) ),np.argmin( np.abs( lon - lon0 ) )
print( z,y,x)


In [ ]:
plt.plot( rho_epwp[:,z,y,x] )
plt.plot( tau_tot_sm[:,z,y,x] )


In [ ]:
"""
%%time
coco=np.zeros( (nz,ny,nx) )
print( nz,ny,nx)
print( rho_epwp.shape)
print( tau_tot_sm.shape)
for z in np.arange( nz ):
    for y in np.arange( ny ):
        for x in np.arange( nx ):
            cox=rho_epwp[:,z,y,x]
            coy=tau_tot_sm[:,z,y,x]
            mask = np.isfinite(cox) & np.isfinite(coy)
            coco[z,y,x] = np.corrcoef(cox[mask], coy[mask])[0,1]

"""

In [ ]:
z0,lat0,lon0 = 20_000., -50.,50.
z,y,x = np.argmin( np.abs( zlev - z0 ) ),np.argmin( np.abs( lat-lat0 ) ),np.argmin( np.abs( lon - lon0 ) )


#plt.contourf(lon,lat,coco[z,:,:] )
#plt.colorbar()

In [ ]:
tau_mmt.shape

In [ ]:
plt.contourf( np.average( tau_mmt, axis=0)[z,:,:] )

In [ ]:
plt.plot( tau_mmt[:,25,112,40] )
#plt.plot( tau_mmt[:,25,20,40] )
print( np.max( tau_mmt ) )
#plt.xlim(20,30)

In [ ]:
tausrc_mm = Xgw.TAU_SRC_MOVMVTN.values
print( list(Xgw.variables ))

In [ ]:
#plt.plot( tau_mmt[:,25,112,40]
tausrc_mm.shape


In [ ]:
z=46
print( zlev[z] )
plt.contourf ( zeta[t,z,:,: ], levels=51 )

In [ ]:
enstro=zeta**2

In [ ]:
slev=1.e-8 * np.linspace( 0.,1.,num=51 )
cmap='gist_ncar'

fig,ax=plt.subplots( figsize=(15,7) )
c = ax.contourf( np.mean( np.mean( enstro[:,46:,:,:], axis=0 ), axis=0) ,levels=slev , cmap=cmap )
t=ax.contour( htopo, levels=[0,1,100] )
ax.set_title( case )
plt.colorbar(c)

In [ ]:
plt.plot( np.average( tau_mmt, axis=0)[:,112,40] )

In [ ]:
plt.plot( np.average( tau_mmt, axis=0)[z,112,:] )

In [ ]:
Xgw


In [ ]:
fig,axs=plt.subplots( 1,2, figsize=(15,7))
p=0
ax=axs[p]
ax.plot(coco[:,y,x],zlev )

p=1
ax=axs[p]
ax.plot(rho_epwp_tv[1:,y,x],zlev )



In [ ]:
plt.plot(lon,coco[z,y,:] )

In [ ]:
plt.scatter( tau_tot_sm_tv[z,:,:], epwp_tv[z,:,:] )
#plt.scatter( tau_tot_tv[z,:,:] , epwp_tv[z,:,:] )
print( zlev[z] )

In [ ]:
Xgw

In [ ]:
print(zlev[z])
print(X.lev[z].values)

In [ ]:
plt.plot( axv[:,30:50,0:100].flatten() )